# RCT replication of results

Main thing to check if Andreas' notebook exists?

Steps:
- Speak to andreas and see if he has the original analysis
1. Download the results (done)
2. Find only the RCT results (partially)
3. Filter for RCT results we trust 
4. Graph the survey responses
5. Save graphs as .pdfs and required anon data into .xlsx for sharing

Download the results

In [2]:
import boto3
import pandas as pd

session = boto3.Session()
credentials = session.get_credentials()

dynamodb = boto3.resource("dynamodb", region_name="eu-west-2")
responses_table = dynamodb.Table("caddyResponses-prod")

all_items = []

response = responses_table.scan()
all_items.extend(response['Items'])
while 'LastEvaluatedKey' in response:
    response = responses_table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    all_items.extend(response['Items'])

df_responses = pd.DataFrame(all_items)
df_responses['Office'] = df_responses.apply(lambda x: str(x['approverEmail']).split('@')[-1], axis=1)
df_responses.shape

(1062, 16)

In [3]:
messages_table = dynamodb.Table("caddyMessages-prod")

all_items = []

message = messages_table.scan()
all_items.extend(message['Items'])
while 'LastEvaluatedKey' in message:
    message = messages_table.scan(ExclusiveStartKey=message['LastEvaluatedKey'])
    all_items.extend(message['Items'])

df_messages = pd.DataFrame(all_items)
df_messages.shape

(1388, 8)

In [4]:
evaluation_table = dynamodb.Table("caddyEvaluation-prod")

all_items = []

evaluation = evaluation_table.scan()
all_items.extend(evaluation['Items'])
while 'LastEvaluatedKey' in evaluation:
    evaluation = evaluation_table.scan(ExclusiveStartKey=evaluation['LastEvaluatedKey'])
    all_items.extend(evaluation['Items'])

# Convert the list of items to a DataFrame
df_evaluation = pd.DataFrame(all_items)
df_evaluation.shape

(1023, 9)

Filtering for only RCT

In [5]:
df_RCT = df_responses[(df_responses['Office'] == 'casort.org') | (df_responses['Office'] == 'citizensadvicemanchester.org.uk')]

In [6]:
df_RCT = pd.merge(left=df_RCT, right=df_evaluation, on='threadId', how='left')

In [7]:
df_RCT = df_RCT[(df_RCT['approvalTimestamp'] >= '2024-04-22') & (df_RCT['approvalTimestamp'] <= '2024-06-21')] # these are the dates the rct ran between

In [8]:
def rctGrouper(message):
    return 'control' if isinstance(message, (dict, list)) else 'treatment'       

df_RCT['Control or Treatment'] = df_RCT.apply(lambda x: rctGrouper(x['modulesUsed']), axis=1) # check whether this is correct?

In [9]:
import pandas as pd

# List of excluded users
excluded_users = [
    'siobhan.durcan@citizensadvice.org.uk', 
    'siobhan.durcan@casort.org.uk',
    'AVarotsis@no10.gov.uk',
    'kurtis.massey@casort.org',
    'siobhan.durcan@casort.org',
    'Chetan.chauhan-sims@ukhsa.gov.uk',
    'hanna.harding@cabinetoffice.gov.uk',
    'max.hollingdale@cabinetoffice.gov.uk',
    'alex.moore@cabinetoffice.gov.uk'
]

df_RCT_filtered = df_RCT[~df_RCT['approverEmail'].isin(excluded_users)]

Graphing of results

- Split out control and treatment groups
- Find approval rate
- Find survey responses

In [14]:
df_responses

,approvalTimestamp,approverEmail,approverReceivedTimestamp,llmResponseTimestamp,userThankedTimestamp,approved,llmPromptTimestamp,messageId,llmAnswer,supervisorMessage,llmResponseJSon,responseId,llmPrompt,threadId,userResponseTimestamp,Office
0,2024-07-29 12:58:52.876802+00:00,sarahm@citizensadvice-doncasterborough.org,2024-07-29 12:56:45.323150,2024-07-29 12:56:40.901607,2024-07-29 12:56:42.142827,True,2024-07-29 12:55:16.317499,9XYRPT0_GZs.9XYRPT0_GZs,"<font color=""#004f88""><b>Brief Summary:</b> To...",None,{'cardsV2': [{'card': {'sections': [{'widgets'...,a393ddba-2ed5-432c-906d-8bbd23c20d99,how to apply for council tax reduction in Donc...,9XYRPT0_GZs,2024-07-29 12:58:58.417204,citizensadvice-doncasterborough.org
1,2024-07-30 14:43:11.404364+00:00,ca_admin4@citizensadvicenorthumberland.org.uk,2024-07-30 14:41:43.737361,2024-07-30 14:41:39.721975,2024-07-30 14:41:41.500323,True,2024-07-30 14:40:16.941867,9UY_6vHKveI.9UY_6vHKveI,"<font color=""#004f88""><b>Brief Summary:</b> Ad...",None,{'cardsV2': [{'card': {'sections': [{'widgets'...,d79bc3de-4e0a-47af-9293-5a7072103fe1,"I have a client that is struggling with debts,...",9UY_6vHKveI,2024-07-30 14:43:16.666752,citizensadvicenorthumberland.org.uk
2,2024-07-10 10:51:42.680234+00:00,sarahm@citizensadvice-doncasterborough.org,2024-07-10 10:50:14.923927,2024-07-10 10:50:10.716039,2024-07-10 10:50:12.348356,True,2024-07-10 10:48:55.466094,SAHP2Y9tCRM.SAHP2Y9tCRM,"<font color=""#004f88""><b>Brief Summary:</b> <i...",None,{'cardsV2': [{'card': {'sections': [{'widgets'...,f39297a7-ec7e-4832-8740-425693a257ae,How do I apply for residence as a family membe...,SAHP2Y9tCRM,2024-07-10 10:51:48.552947,citizensadvice-doncasterborough.org
3,2024-07-04 10:10:52.952145+00:00,momna.azad@citizensadvicemanchester.org.uk,2024-07-04 10:10:48.804948,2024-07-04 10:10:45.857539,2024-07-04 10:10:46.878537,False,2024-07-04 10:09:51.964797,ogTQz8PHcg8.ogTQz8PHcg8,"<font color=""#004f88""><b>Brief Summary:</b> Yo...",,{'cardsV2': [{'card': {'sections': [{'widgets'...,1a9aece4-fe96-4ee2-a8a1-8344a787331b,can i get information from you?,ogTQz8PHcg8,2024-07-04 10:10:57.514419,citizensadvicemanchester.org.uk
4,2024-07-01 10:14:35.445705+00:00,ayesha.qudsia@casort.org,2024-07-01 10:12:21.576727,2024-07-01 10:12:17.154050,2024-07-01 10:12:18.961727,True,2024-07-01 10:10:42.207175,3kPiyGnCrRo.3kPiyGnCrRo,"<font color=""#004f88""><b>Brief Summary:</b> An...",None,{'cardsV2': [{'card': {'sections': [{'widgets'...,7cdab9db-0c56-43b6-b730-d6360a364ccc,I am going through an employment tribunal but...,3kPiyGnCrRo,2024-07-01 10:14:42.076737,casort.org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,2024-08-14 13:45:27.491999+00:00,siobhan.durcan@casort.org,2024-08-14 13:44:25.554055,2024-08-14 13:44:21.561876,2024-08-14 13:44:22.772072,True,2024-08-14 13:43:14.647179,CxnQ2EuYvmU.857IgF5Qzls,"<font color=""#004f88""><b>Brief Summary:</b> <i...",None,{'cardsV2': [{'card': {'sections': [{'widgets'...,29b9ecc0-436e-41d6-805f-b7a4b9844a2b,Can my client return an item that they bought...,CxnQ2EuYvmU,2024-08-14 13:45:33.478725,casort.org
1058,2024-05-14 10:44:02.655898+00:00,chiefofficer@doverdealcitizensadvice.org.uk,2024-05-14 10:30:26.315678,2024-05-14 10:30:21.888327,2024-05-14 10:30:26.321683,True,2024-05-14 10:29:41.871326,2nbW3PLBD1k.2nbW3PLBD1k,"<font color=""#004f88""><b>TL;DR:</b> An evictio...",None,"{""cardsV2"": [{""cardId"": ""aiResponseCard"", ""car...",927c4312-ba55-49e5-98d9-919acbac73b7,If I am in the process of paying a DRO will an...,2nbW3PLBD1k,2024-05-14 10:44:08.734945,doverdealcitizensadvice.org.uk
1059,2024-06-06 14:16:42.441278+00:00,momna.azad@citizensadvicemanchester.org.uk,2024-06-06 14:15:05.645162,2024-06-06 14:15:02.538895,2024-06-06 14:15:03.477881,True,2024-06-06 14:14:02.984327,nv49Kd-d1BM.nv49Kd-d1BM,"<font color=""#004f88""><b>Brief Summary:</b> Th...",None,{'cardsV2': [{'card': {'sections': [{'widgets'...,e3394a41-2fd6-48af-b103-d5a4f5593dfa,the client was a stu

In [10]:
df_RCT_filtered_control = df_RCT_filtered[df_RCT_filtered['Control or Treatment'] == 'control']
df_RCT_filtered_treatment = df_RCT_filtered[df_RCT_filtered['Control or Treatment'] == 'treatment']

df_RCT_approval = df_RCT_filtered_control[['approved', 'approvalTimestamp']].groupby(by='approved').count()
df_RCT_approval['percentage'] = (df_RCT_approval['approvalTimestamp'] / df_RCT_approval['approvalTimestamp'].sum()) * 100
df_RCT_approval


,approvalTimestamp,percentage
approved,,
False,37,22.155689
True,130,77.844311


In [11]:
df_RCT['controlGroupMessage']

2       NaN
3      None
4       NaN
6      None
7      None
       ... 
457    None
458    None
459     NaN
462     NaN
464    None
Name: controlGroupMessage, Length: 298, dtype: object

In [12]:
df_RCT_filtered_control

,approvalTimestamp,approverEmail,approverReceivedTimestamp,llmResponseTimestamp,userThankedTimestamp,approved,llmPromptTimestamp,messageId,llmAnswer,supervisorMessage,...,Office,callComplete,surveyResponse,continueConversation,controlGroupMessage,moduleOutputs,callStart,modulesUsed,surveyCompleteTimestamp,Control or Treatment
3,2024-05-24 11:12:40.044902+00:00,jamie.walker@citizensadvicemanchester.org.uk,2024-05-24 10:52:19.442129,2024-05-24 10:52:16.268586,2024-05-24 10:52:17.334049,True,2024-05-24 10:50:57.547456,_FJRGMey1mU._FJRGMey1mU,"<font color=""#004f88""><b>Brief Summary:</b> Fo...",None,...,citizensadvicemanchester.org.uk,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",24-05-2024 11:50,"{'module_arguments': {'split': '0.5', 'control...",24-05-2024 12:16:35,control
6,2024-04-25 13:12:47.138140+00:00,jamie.mcglynn@casort.org,2024-04-25 13:12:12.333274,2024-04-25 13:12:09.570428,2024-04-25 13:12:12.340011,True,2024-04-25 13:11:23.807528,4BdewiS6-ww.ymMqb6qE1DY,"Based on the information provided, it seems yo...",None,...,casort.org,True,NaN,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",NaN,"{'module_arguments': {'split': '0.5', 'control...",NaN,control
7,2024-05-31 08:41:21.197971+00:00,tom.hall@casort.org,2024-05-31 08:41:09.203845,2024-05-31 08:41:05.406203,2024-05-31 08:41:06.641394,False,2024-05-31 08:39:54.079991,CLLLZRrrryw.wvFbbwsKm84,"<font color=""#004f88""><b>Brief Summary:</b> Yo...",,...,casort.org,False,NaN,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",31-05-2024 09:39,"{'module_arguments': {'split': '0.5', 'control...",NaN,control
9,2024-06-19 15:05:25.457493+00:00,aly.darwin@casort.org,2024-06-19 15:04:41.197436,2024-06-19 15:04:36.667270,2024-06-19 15:04:38.606880,True,2024-06-19 15:03:42.119082,aRimZe6WqPM.aRimZe6WqPM,"<font color=""#004f88""><b>Brief Summary:</b> <i...",None,...,casort.org,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",19-06-2024 16:03,"{'module_arguments': {'split': '0.5', 'control...",19-06-2024 16:07:20,control
10,2024-05-14 13:27:28.612922+00:00,melanie.chapman@casort.org,2024-05-14 13:25:35.858018,2024-05-14 13:25:33.150492,2024-05-14 13:25:35.864541,False,2024-05-14 13:25:01.409361,6FVnE5lPZzo.MSgJZZVzp1g,"<font color=""#004f88""><b>TL;DR</b> <i>Yes, Uni...","The advice is correct, but does not match what...",...,casort.org,False,NaN,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",NaN,"{'module_arguments': {'split': '0.5', 'control...",NaN,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,2024-06-11 08:55:34.487217+00:00,greg.sammons@citizensadvicemanchester.org.uk,2024-06-11 08:54:19.805747,2024-06-11 08:54:17.014193,2024-06-11 08:54:18.010815,True,2024-06-11 08:53:21.874621,GA_95vYrkVc.GA_95vYrkVc,"<font color=""#004f88""><b>Brief Summary:</b> Yo...",None,...,citizensadvicemanchester.org.uk,True,[{'How confident did you feel in providing adv...,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",11-06-2024 09:53,"{'module_arguments': {'split': '0.5', 'control...",11-06-2024 14:35:34,control
453,2024-04-30 13:49:59.081554+00:00,mohammed.rahman@citizensadvicemanchester.org.uk,2024-04-30 13:48:14.561132,2024-04-30 13:48:10.639044,2024-04-30 13:48:14.567696,False,2024-04-30 13:47:37.621448,Pp46f7K7Erg.Pp46f7K7Erg,"Based on the information provided, here is my ...","Hey Jeevan, more detail will be required to ge...",...,citizensadvicemanchester.org.uk,False,NaN,True,None,"{""randomisation"": [""continue_interaction"", ""tr...",NaN,"{'module_arguments': {'split': '0.5', 'control...",NaN,control
456,2024-05-14 13:12:13.820440+00:00,melanie.chapman@casort.org,2024-05-14 13:10:05.888132,2024-05-14 13:10:03.019838,2024-05-14 13:10:05.894695,True,2024-05-14 13:09:30.695910,tiSdwRA3Bgo.2GkCJ_OPRLQ,"<font color=""#004f88"

In [13]:
df_evaluation['modulesUsed'][2]['module_arguments']

{'split': '0',
 'control_group_message': 'Apologies, for this call you will not have access to Caddy due to the randomised control trial.\n\nCaddy will be available for future calls but for now you can send your query to the supervisor using the button below.\n\n<b> Please fill in the survey about your experience answering this call, regardless of the support you receive.</b>'}